Latex Macros:
$$
\newcommand{\E}{\text{E}}
\newcommand{\mbf}{\mathbf}
\newcommand{\bs}{\boldsymbol}
\newcommand{\Cov}{\text{Cov}}
\newcommand{\Var}{\text{Var}}
\newcommand{\Cov}{\text{Cov}}
\newcommand{\A}[1]{\mathbf{A}_{#1}}
\newcommand{\Ai}[1]{\mathbf{A}^{#1}}
$$

# Inference on difference beteween the most favorable and least unfavorable genotypic values  

Consider the model:
$$
\mbf{y} = \mbf{1}\bs{\mu} + \mbf{M}\bs{\alpha} + \mbf{e},
$$

where $\mbf{M}$ is a matrix of SNP covariates coded as 0, 1 or 2 (number of copies of the "A" allele). At locus $j$, if the "A" allele is favorable, $\alpha_j$ is positive, and if it is unfavorable,$\alpha_j$ is negative. So, if locus $j$ is favorable, the difference between the favorable and unfavorable homozygotes will be $2 \times \alpha_j$. On the other hand, if locus $j$ is unfavorable, this difference becomes $2\times -\alpha_j$. Thus, at an arbitrary locus, the difference between the favorable and unfavorable homozygotes is $2 \times |\alpha_j|$, and across all the loci, the difference between the most favorable and least favorable genotypic values is:

$$
D = 2\sum_j |\alpha_j|.
$$

But, as the $\bs{\alpha}$ is not observed, $D$ can be estimated as
$$
D = 2\sum_j |\hat{\alpha}_j|.
$$

Below are results from several estimators of $\bs{\alpha}$. 

In [1]:
using Statistics, Distributions, JWAS, LinearAlgebra, Printf, Plots, CSV, DataFrames
using Random

Here is a simple way to define a function:

In [2]:
Identity(n) = Matrix(I,n,n)

Identity (generic function with 1 method)

In [3]:
function getEstimates(X,mmei,xpxi,probNull, Vg, Ve)
    n,p = size(X)
    pNull = Int(probNull*p)
    pNonNull = p - pNull
    Valpha = 2Vg/p
    α = [randn(pNonNull)*sqrt(Valpha); zeros(pNull)]
    y = X*α + randn(n) .* sqrt(Ve)
    data = DataFrame([string.(1:n) y])
    names!(data,[:ind, :y])
    
    model = build_model("y = intercept", Ve)
    add_genotypes(model,"genotypes", Vg; G_is_marker_variance=false, separator=',',header=false)
    
    ### Bayes A ###
    outputA = runMCMC(model,data;
                Pi                  = 0.0,
                burnin              = 1000,
                chain_length        = 11000,
                starting_value      = false,
                missing_phenotypes  = false,
                constraint          = false,
                estimatePi          = false,
                estimate_variance   = true,
                methods             = "BayesB",
                printout_frequency  = 5_000,
                printout_model_info = false,
                output_samples_frequency = 0,
                update_priors_frequency=0,
                #parameters for single-step analysis
                single_step_analysis= false,
                pedigree            = false,
                #output
                #Genomic Prediction
                outputEBV               = false,
                output_genetic_variance = true,
                output_residual_variance= true,
                output_heritability     = false
    )
    
    ### Bayes BPi ###
    model = build_model("y = intercept", Ve)
    add_genotypes(model,"genotypes", Vg; G_is_marker_variance=false, separator=',',header=false)
    outputB = runMCMC(model,data;
                Pi                  = 0.5,
                burnin              = 1000,
                chain_length        = 11000,
                starting_value      = false,
                missing_phenotypes  = false,
                constraint          = false,
                estimatePi          = true,
                estimate_variance   = true,
                methods             = "BayesB",
                printout_frequency  = 5_000,
                printout_model_info = false,
                output_samples_frequency = 0,
                update_priors_frequency=0,
                #parameters for single-step analysis
                single_step_analysis= false,
                pedigree            = false,
                #output
                #Genomic Prediction
                outputEBV               = false,
                output_genetic_variance = true,
                output_residual_variance= true,
                output_heritability     = false
    )
    
    ### Bayes CPi ###
    model = build_model("y = intercept", Ve)
    add_genotypes(model,"genotypes", Vg; G_is_marker_variance=false, separator=',',header=false)
    outputC = runMCMC(model,data;
                Pi                  = 0.5,
                burnin              = 1000,
                chain_length        = 11000,
                starting_value      = false,
                missing_phenotypes  = false,
                constraint          = false,
                estimatePi          = true,
                estimate_variance   = true,
                methods             = "BayesC",
                printout_frequency  = 5_000,
                printout_model_info = false,
                output_samples_frequency = 0,
                update_priors_frequency=0,
                #parameters for single-step analysis
                single_step_analysis= false,
                pedigree            = false,
                #output
                #Genomic Prediction
                outputEBV               = false,
                output_genetic_variance = true,
                output_residual_variance= true,
                output_heritability     = false
    )
    ### Bayes C0 ###
    model = build_model("y = intercept", Ve)
    add_genotypes(model,"genotypes", Vg; G_is_marker_variance=false, separator=',',header=false)
    outputC0 = runMCMC(model,data;
                Pi                  = 0.0,
                burnin              = 1000,
                chain_length        = 11000,
                starting_value      = false,
                missing_phenotypes  = false,
                constraint          = false,
                estimatePi          = false,
                estimate_variance   = true,
                methods             = "BayesC",
                printout_frequency  = 5_000,
                printout_model_info = false,
                output_samples_frequency = 0,
                update_priors_frequency=0,
                #parameters for single-step analysis
                single_step_analysis= false,
                pedigree            = false,
                #output
                #Genomic Prediction
                outputEBV               = false,
                output_genetic_variance = true,
                output_residual_variance= true,
                output_heritability     = false
    )
      
    rhs = X'y
    
    ### OLS ###
    ols  = xpxi*rhs
    
    bayesa  = outputA["Posterior mean of marker effects"]
    bayesb  = outputB["Posterior mean of marker effects"]
    bayesc  = outputC["Posterior mean of marker effects"]
    blup    = outputC0["Posterior mean of marker effects"]
    
    trueMean   = 2*sum(abs.(α))
    bayesaMean = 2*sum(abs.(bayesa))
    bayesbMean = 2*sum(abs.(bayesb))
    bayescMean = 2*sum(abs.(bayesc))
    blupMean   = 2*sum(abs.(blup))
    olsMean    = 2*sum(abs.(ols))    
    return trueMean,bayesaMean,bayesbMean,bayescMean,blupMean,olsMean
end

getEstimates (generic function with 1 method)

### Comparison of BayesA, BayesB, BayesCPi, RR-BLUP and OLS with Sample size n = 100 and number p = 20 of markers and $\pi = 0.75$

In [4]:
n  = 100
p  = 20
probNull = 0.75

N  = 25
Vg = 20
Ve = 10

Random.seed!(1234)

Valpha = Vg/p*2
λ = Ve/Valpha
X = rand(Binomial(2,0.5), n,p)
mme = float(X'X + Identity(p)*λ)
mmei = inv(mme)
xpxi = inv(X'X)
CSV.write("genotypes",DataFrame([string.(1:n) X]), writeheader=false)

resSim = [getEstimates(X,mmei,xpxi,probNull,Vg, Ve) for i=1:N]
resMat = [row[i] for row in resSim, i=1:6]

resMeans = mean(resMat,dims=1)
resStd = std(resMat,dims=1)./N;

┌ Warning: readtable is deprecated, use CSV.read from the CSV package instead
│   caller = ip:0x0
└ @ Core :-1


20 markers on 100 individuals were added.

The prior for marker effects variance is calculated from the genetic variance and π. The mean of the prior for the marker effects variance is: 2.008254




running MCMC for BayesB...  0%|                         |  ETA: 1:38:33


Posterior means at iteration: 5000
Residual variance: 9.123002

Posterior means at iteration: 10000
Residual variance: 9.159126


running MCMC for BayesB...100%|█████████████████████████| Time: 0:00:01


...

running MCMC for BayesB...100%|█████████████████████████| Time: 0:00:00



Posterior means at iteration: 10000
Residual variance: 10.409145
Marker effects variance: 2.117755
π: 0.561
20 markers on 100 individuals were added.

The prior for marker effects variance is calculated from the genetic variance and π. The mean of the prior for the marker effects variance is: 2.008254




running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:00



Posterior means at iteration: 5000
Residual variance: 10.394997
Marker effects variance: 0.782202

Posterior means at iteration: 10000
Residual variance: 10.375944
Marker effects variance: 0.784395


running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:00


In [5]:
println("     True   BayesA BayesB BayesC RR-BLUP  OLS")
@printf("    %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f\n", resMeans[1],resMeans[2],resMeans[3],resMeans[4],resMeans[5],resMeans[6] )
@printf("+/- %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f\n", resStd[1],resStd[2],resStd[3],resStd[4],resStd[5],resStd[6] )

     True   BayesA BayesB BayesC RR-BLUP  OLS
     10.76  18.66  10.96  11.10  16.20  23.02
+/-   0.12   0.12   0.19   0.18   0.14   0.15


### Comparison of BayesA, BayesB, BayesCPi, RR-BLUP and OLS with Sample size n = 100 and number p = 1000 of markers and $\pi=0.95$

In [6]:
n  = 100
p  = 1000
probNull = 0.95

N  = 25
Vg = 20
Ve = 10

Random.seed!(4321)

Valpha = Vg/p*2
λ = Ve/Valpha
X = rand(Binomial(2,0.5), n,p)
mme = float(X'X + Identity(p)*λ)
mmei = inv(mme)
xpxi = inv(X'X + Identity(p)*.001)
CSV.write("genotypes",DataFrame([string.(1:n) X]), writeheader=false)

resSim = [getEstimates(X,mmei,xpxi,probNull,Vg, Ve) for i=1:N]
resMat = [row[i] for row in resSim, i=1:6]

resMeans = mean(resMat,dims=1)
resStd = std(resMat,dims=1)./N;

1000 markers on 100 individuals were added.

The prior for marker effects variance is calculated from the genetic variance and π. The mean of the prior for the marker effects variance is: 0.040202




running MCMC for BayesB... 41%|██████████               |  ETA: 0:00:03


Posterior means at iteration: 5000
Residual variance: 2.397599


running MCMC for BayesB... 87%|██████████████████████   |  ETA: 0:00:01


Posterior means at iteration: 10000
Residual variance: 2.360425


...

running MCMC for BayesC... 44%|███████████              |  ETA: 0:00:02


Posterior means at iteration: 5000
Residual variance: 3.694373
Marker effects variance: 0.010017


running MCMC for BayesC... 84%|█████████████████████    |  ETA: 0:00:01


Posterior means at iteration: 10000
Residual variance: 3.728024
Marker effects variance: 0.009847


running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:-1


In [7]:
println("     True   BayesA BayesB BayesC RR-BLUP  OLS")
@printf("    %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f\n", resMeans[1],resMeans[2],resMeans[3],resMeans[4],resMeans[5],resMeans[6] )
@printf("+/- %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f\n", resStd[1],resStd[2],resStd[3],resStd[4],resStd[5],resStd[6] )

     True   BayesA BayesB BayesC RR-BLUP  OLS
     15.91  65.22  34.22  31.02  41.70  78.04
+/-   0.07   0.14   0.41   0.38   0.22   0.22


### Comparison of BayesA, BayesB, BayesCPi, RR-BLUP and OLS with Sample size n = 300 and number p = 1000 of markers and $\pi = 0.95$

In [8]:
n  = 300
p  = 1000
probNull = 0.95

N  = 25
Vg = 20
Ve = 10

Random.seed!(4321)

Valpha = Vg/p*2
λ = Ve/Valpha
X = rand(Binomial(2,0.5), n,p)
mme = float(X'X + Identity(p)*λ)
mmei = inv(mme)
xpxi = inv(X'X)
CSV.write("genotypes",DataFrame([string.(1:n) X]), writeheader=false)

resSim = [getEstimates(X,mmei,xpxi,probNull,Vg, Ve) for i=1:N]
resMat = [row[i] for row in resSim, i=1:6]

resMeans = mean(resMat,dims=1)
resStd = std(resMat,dims=1)./N;

1000 markers on 300 individuals were added.

The prior for marker effects variance is calculated from the genetic variance and π. The mean of the prior for the marker effects variance is: 0.040065




running MCMC for BayesB... 44%|███████████              |  ETA: 0:00:04


Posterior means at iteration: 5000
Residual variance: 2.427125


running MCMC for BayesB... 88%|██████████████████████   |  ETA: 0:00:01


Posterior means at iteration: 10000
Residual variance: 2.410644


...

running MCMC for BayesC... 41%|██████████               |  ETA: 0:00:03


Posterior means at iteration: 5000
Residual variance: 6.626966
Marker effects variance: 0.009807


running MCMC for BayesC... 86%|█████████████████████    |  ETA: 0:00:01


Posterior means at iteration: 10000
Residual variance: 6.693858
Marker effects variance: 0.009636


running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:05


In [9]:
println("     True   BayesA BayesB BayesC RR-BLUP  OLS")
@printf("    %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f\n", resMeans[1],resMeans[2],resMeans[3],resMeans[4],resMeans[5],resMeans[6] )
@printf("+/- %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f\n", resStd[1],resStd[2],resStd[3],resStd[4],resStd[5],resStd[6] )

     True   BayesA BayesB BayesC RR-BLUP  OLS
     16.11 114.84  25.60  25.31  45.97 3199.37
+/-   0.05   0.14   0.54   0.43   0.30  41.79


### Comparison of BayesA, BayesB, BayesCPi, RR-BLUP and OLS with Sample size n = 500 and number p = 1000 of markers and $\pi=0.95$

In [10]:
n  = 500
p  = 1000
probNull = 0.95

N  = 25
Vg = 20
Ve = 10

Random.seed!(4321)

Valpha = Vg/p*2
λ = Ve/Valpha
X = rand(Binomial(2,0.5), n,p)
mme = float(X'X + Identity(p)*λ)
mmei = inv(mme)
xpxi = inv(X'X)
CSV.write("genotypes",DataFrame([string.(1:n) X]), writeheader=false)

resSim = [getEstimates(X,mmei,xpxi,probNull,Vg, Ve) for i=1:N]
resMat = [row[i] for row in resSim, i=1:6]

resMeans = mean(resMat,dims=1)
resStd = std(resMat,dims=1)./N;

1000 markers on 500 individuals were added.

The prior for marker effects variance is calculated from the genetic variance and π. The mean of the prior for the marker effects variance is: 0.04004




running MCMC for BayesB... 44%|███████████              |  ETA: 0:00:06


Posterior means at iteration: 5000
Residual variance: 5.791029


running MCMC for BayesB... 90%|██████████████████████   |  ETA: 0:00:01


Posterior means at iteration: 10000
Residual variance: 5.733718


...

running MCMC for BayesC... 43%|███████████              |  ETA: 0:00:04


Posterior means at iteration: 5000
Residual variance: 8.536633
Marker effects variance: 0.00593


running MCMC for BayesC... 88%|██████████████████████   |  ETA: 0:00:01


Posterior means at iteration: 10000
Residual variance: 8.583718
Marker effects variance: 0.005769


running MCMC for BayesC...100%|█████████████████████████| Time: 0:00:07


In [11]:
println("     True   BayesA BayesB BayesC RR-BLUP  OLS")
@printf("    %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f\n", resMeans[1],resMeans[2],resMeans[3],resMeans[4],resMeans[5],resMeans[6] )
@printf("+/- %6.2f %6.2f %6.2f %6.2f %6.2f %6.2f\n", resStd[1],resStd[2],resStd[3],resStd[4],resStd[5],resStd[6] )

     True   BayesA BayesB BayesC RR-BLUP  OLS
     16.32 130.39  17.39  20.92  44.72 1946637.60
+/-   0.07   0.17   0.31   0.34   0.26 52874.12
